<a href="https://colab.research.google.com/github/jlopetegui98/Creation-of-a-synthetic-dataset-for-French-NER-in-clinical-trial-texts/blob/main/TranslationModel/evaluate_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# uncomment the following line to run in colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 1.2 MB/s eta 0:00:00


In [2]:
# uncomment if using colab
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U datasets
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U evaluate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.1 MB/s eta 0:00:00


In [3]:
import evaluate
import transformers
import os
import json

In [14]:
# paths to the data
# root_path = '..' # comment if using colab
root_path = './drive/MyDrive/HandsOn-NLP/' # uncomment if using colab
data_path = f'{root_path}/data'
french_texts_path = f'{data_path}/chia_criteria_french'
chat_gpt_translations_path = f'{data_path}/chia_criteria_french_chat_gpt'
t5_translations_path = f'{data_path}/chia_criteria_french_t5'

In [15]:
# load files used for evaluation
files = os.listdir(t5_translations_path)

In [27]:
all_sentences = []
all_sentences_t5 = []

for file in files:
    with open(f'{french_texts_path}/{file}', 'r') as f:
        sentences = [line for line in f.readlines() if (line.strip() != '' and line.strip() != '\n' and len(line.split()) < 500) and len(line.split()) >= 1]
    with open(f'{t5_translations_path}/{file}', 'r') as f:
        sentences_t5 = [line for line in f.readlines() if (line.strip() != '' and line.strip() != '\n' and len(line.split()) < 500) and len(line.split()) >= 1]
    for i in range(len(sentences)):
        if len(sentences[i].split()) < 2 or len(sentences_t5[i].split()) < 2:
          continue
        all_sentences.append(sentences[i])
        all_sentences_t5.append(sentences_t5[i])
    if len(sentences) != len(sentences_t5):
        print(file)

In [28]:
len(all_sentences), len(all_sentences_t5)

(562, 562)

**Evaluate perplexity for the model used in translations: Helsinki-NLP/opus-mt-en-fr**

In [30]:
perplexity = evaluate.load("perplexity", module_type="metric")

results = perplexity.compute(model_id='Helsinki-NLP/opus-mt-en-fr',
                             add_start_token=False,
                             predictions=all_sentences)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


  0%|          | 0/36 [00:00<?, ?it/s]

In [12]:
with open(f'{data_path}/perplexity_results.json', 'w') as f:
    json.dump(results, f)

In [31]:
results

{'perplexities': [2767.919677734375,
  3882.803466796875,
  5924.3525390625,
  2668.077392578125,
  3511.810546875,
  7572.55859375,
  5450.00390625,
  19883.9453125,
  3232.912109375,
  2404.91357421875,
  33411.28125,
  4324.07275390625,
  4135.71875,
  10248.9453125,
  17318.42578125,
  15298.943359375,
  8263.8330078125,
  3070.6982421875,
  2510.333251953125,
  4591.447265625,
  9625.0732421875,
  5835.56396484375,
  2043.77099609375,
  11369.9033203125,
  18313.9140625,
  12417.08203125,
  5757.64697265625,
  3120.17041015625,
  7309.134765625,
  4009.886962890625,
  3443.048583984375,
  17459.6328125,
  4750.650390625,
  1764.1854248046875,
  617.107421875,
  6885.32568359375,
  50230.61328125,
  8752.419921875,
  9234.546875,
  4059.987548828125,
  19092.736328125,
  8542.400390625,
  31784.787109375,
  27625.05859375,
  35867.265625,
  1890.568603515625,
  1249.9429931640625,
  3087.9482421875,
  10904.8271484375,
  10909.2265625,
  4262.95361328125,
  5626.00390625,
  4791.12

**Evaluate perplexity for t5 model**

In [32]:
results_t5 = perplexity.compute(model_id='google-t5/t5-small',
                             add_start_token=False,
                             predictions=all_sentences_t5)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

ValueError: Unrecognized configuration class <class 'transformers.models.t5.configuration_t5.T5Config'> for this kind of AutoModel: AutoModelForCausalLM.
Model type should be one of BartConfig, BertConfig, BertGenerationConfig, BigBirdConfig, BigBirdPegasusConfig, BioGptConfig, BlenderbotConfig, BlenderbotSmallConfig, BloomConfig, CamembertConfig, LlamaConfig, CodeGenConfig, CpmAntConfig, CTRLConfig, Data2VecTextConfig, ElectraConfig, ErnieConfig, FalconConfig, FuyuConfig, GemmaConfig, GitConfig, GPT2Config, GPT2Config, GPTBigCodeConfig, GPTNeoConfig, GPTNeoXConfig, GPTNeoXJapaneseConfig, GPTJConfig, LlamaConfig, MarianConfig, MBartConfig, MegaConfig, MegatronBertConfig, MistralConfig, MixtralConfig, MptConfig, MusicgenConfig, MvpConfig, OpenLlamaConfig, OpenAIGPTConfig, OPTConfig, PegasusConfig, PersimmonConfig, PhiConfig, PLBartConfig, ProphetNetConfig, QDQBertConfig, Qwen2Config, ReformerConfig, RemBertConfig, RobertaConfig, RobertaPreLayerNormConfig, RoCBertConfig, RoFormerConfig, 

In [ ]:
with open(f'{data_path}/perplexity_results_gpt2.json', 'w') as f:
    json.dump(results_gpt2, f)

**BLEU evaluation**

We are going to use some translations from *chatGPT* as reference to get the BLEU score for the translations used in our task.

In [ ]:
# # get chat_gpt translations
# files_gpt = os.listdir(chat_gpt_translations_path)

# all_sentences_gpt = []
# all_sentences_to_evaluate = []
# for file in files_gpt:
#     with open(f'{chat_gpt_translations_path}/{file}', 'r') as f:
#         all_sentences_gpt.extend([line for line in f.readlines() if (line.strip() != '' and line.strip() != '\n' and len(line.split()) < 500) and len(line.split()) >= 3])
#     with open(f'{french_texts_path}/{file}', 'r') as f:
#         all_sentences_to_evaluate.extend([line for line in f.readlines() if (line.strip() != '' and line.strip() != '\n' and len(line.split()) < 500) and len(line.split()) >= 3])

In [34]:
bleu = evaluate.load("bleu")
results_bleu = bleu.compute(predictions=all_sentences, references=all_sentences_t5)

In [35]:
results_bleu

{'bleu': 0.5269812474607563,
 'precisions': [0.732744340143567,
  0.5725891910985518,
  0.4696759551128483,
  0.39136924955896324],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0384174311926606,
 'translation_length': 9055,
 'reference_length': 8720}

In [36]:
# save results
with open(f'{data_path}/bleu_results.json', 'w') as f:
    json.dump(results_bleu, f)

**TER(Translation Edit Rate) evaluation**

We will use the same approach as for BLEU score case. (using *chatGPT* translations as reference)

In [39]:
ter = evaluate.load("ter")
results_ter = ter.compute(predictions=all_sentences, references=all_sentences_t5)

In [40]:
results_ter

{'score': 39.56783563584839, 'num_edits': 3351, 'ref_length': 8469.0}

In [41]:
# save results
with open(f'{data_path}/ter_results.json', 'w') as f:
    json.dump(results_ter, f)

**METEOR (Metric for Evaluation of Translation with Explicit ORdering)**

We will use the same approach as for BLEU score case. (using *chatGPT* translations as reference)

In [45]:
meteor = evaluate.load("meteor")
results_meteor = meteor.compute(predictions=all_sentences, references=all_sentences_t5)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [46]:
results_meteor

{'meteor': 0.7354905437885116}

In [47]:
# save results
with open(f'{data_path}/meteor_results.json', 'w') as f:
    json.dump(results_meteor, f)